# SocialAL Model
# Fit model to data - one subject
KLS 9.19.19 updated 8.31.20
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

Add decay parameter from : 
Collins, A. G. E., & Frank, M. J. (2012). How much of reinforcement learning is working memory, not reinforcement learning? A behavioral, computational, and neurogenetic analysis. European Journal of Neuroscience, 35(7), 1024–1035. https://doi.org/10.1111/j.1460-9568.2011.07980.x


### Python Version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

### Define functions

In [3]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)
            ev[trustee] = update_value(prob[trustee])
            
            # decay prob for other possible trustees - added 8.31.20
            options = [0, 1, 2]
            options.pop(trustee)
            for x in options:
                prob[x] = prob[x] + decay * (0.5 - prob[x])
        
    return totalLLH

def model_fit_once(data):
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0, 1)
        params = [a_gain, a_loss, beta, decay]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 100), (0, 1)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

def model_fit(data):
    
    tries = 10 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0, 1)
        params = [a_gain, a_loss, beta, decay]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 20), (0,1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
            
    return bestFit
    

### Load and clean data

In [4]:
#os.listdir('../data/modeling') 

In [5]:
#dt = pd.read_csv('../data/modeling/sub-1004.csv')
#dt = pd.read_csv('../data/modeling/sub-1008.csv') #- try getting this sub to work
dt = pd.read_csv('../data/modeling/sub-2032.csv') #- try getting this sub to work
# recode trial type into numbers for model
def stims(trial_type):
    if trial_type == "Trustworthy":
        return 0
    elif trial_type == "Neutral":
        return 1
    elif trial_type == "Untrustworthy":
        return 2
dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
# rename response_key to choice
def choices(response_key):
    if response_key == 'None':
        return 0 
    else:
        return response_key  
dt['Choice'] = dt['response_key'].apply(choices)
dt['Choice'] = pd.to_numeric(dt['Choice'])
# calculte the trustee response
def resp(trial_earnings):
    if trial_earnings >= 12:
        return 1
    else:
        return 0
dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]

In [6]:
getcontext().prec = 10000000 # increases the precision of data representation 
print(getcontext())
params = model_fit(data)
params


Context(prec=10000000, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


      fun: 53.03875771055613
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.07965217e-02,  2.30171921e+02, -4.26325641e-05, -1.08166276e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 680
      nit: 100
   status: 0
  success: True
        x: array([7.66625873e-03, 0.00000000e+00, 1.91809477e+01, 1.00000000e+00])

In [7]:
line = np.append(params.x, params.fun)
line

array([7.66625873e-03, 0.00000000e+00, 1.91809477e+01, 1.00000000e+00,
       5.30387577e+01])